### Muhammad Rizwan Khalid
### BSCS - 6A
### 180459

In [10]:
from PIL import Image, ImageDraw
import numpy as np
import math
img = Image.open("test.jpg").convert("1")
#img.show()

### Our main task is to divide the signature into 64 segments based on the centroids

In [11]:
def bounding_box(img):
    left, top = img.size
    width, height = img.size
    right, bottom = 0,0
    for (x,y) in [(x,y) for x in range(width) for y in range(height)]:
        color = img.getpixel((x, y))
        if color is 0:
            if x > right:
                right = x
            if x < left:
                left = x
            if y > bottom:
                bottom = y
            if y < top:
                top = y
    return left, right, top, bottom    

In [12]:
def find_centroid(img, left, right, top,bottom):
    cx = left
    cy = top
    n = 1
    for (x,y) in [(x,y) for x in range(abs(left-right)) for y in range(abs(top-bottom))]:
        color = img.getpixel((x+left, y+top))
        if color is 0:
            cx += left + x
            cy += top + y
            n += 1
    if(n>1):
        n -= 1
    cx //=  n
    cy //= n
    return cx,cy

### Finding black to white transition using 4 + d neighbourhood

In [13]:
def find_transition(img, left, right, top, bottom):
    n = 0
    for (x,y) in [(x,y) for x in range(left+1,right-1) for y in range(top+1,bottom-1)]:
        pixel = img.getpixel((x, y))
        if(pixel == 255):
            for (i,j) in [(i,j) for i in range(x-1,x+1) for j in range(y-1,y+1)]:
                if(i != j):
                    pixel1 = img.getpixel((i, j)) 
                    if(pixel1 == 0):
                        n = n+1
    return n

### Finding the aspect ratio of every segment

In [14]:
def find_ratio(left, right, top, bottom):
    width =  abs(right - left)
    height = abs(top - bottom)
    if(height>0):
        r = width / height
    else:
        r = 0
    return r

In [15]:
left, right, top, bottom = bounding_box(img)
print(bounding_box(img))

(130, 735, 45, 556)


### Dividing the signature into 8 seegments and then saving the values of centroid, transition and aspect ratio as the features for the signature

In [16]:
def normalize(left, right, top, bottom):
    count = 0
    width = abs(left-right)
    height = abs(top-bottom)
    size  = width * height
    for (x,y) in [(x,y) for x in range(abs(left-right)) for y in range(abs(top-bottom))]:
        color = img.getpixel((x+left, y+top))
        if color is 0:
            count += 1
    if count == 0:
        count = 1
    return float((size/count)) 

In [19]:
t = []
r = []
c = []
norm = []
ang = []
def split(img, left, right, top, bottom, depth=0):
    cx, cy = find_centroid(img, left, right, top, bottom)
    if depth < 3:
        split(img, left, cx, top, cy, depth + 1)
        split(img, cx, right, top, cy, depth + 1)
        split(img, left, cx, cy, bottom, depth + 1)
        split(img, cx, right, cy, bottom, depth + 1)
    else:
        t.append(find_transition(img, left, right, top, bottom))
        r.append(find_ratio(left, right, top, bottom))
        c.append(str(cx)+ " " +str(cy))
        norm.append(normalize(left, right, top, bottom))
        dx, dy = abs(right-cx), abs(bottom-cy)
        ang.append(math.degrees(math.atan2(dy,dx)))
        draw = ImageDraw.Draw(img)
        draw.rectangle(((left, top), (right, bottom)))

In [20]:
split(img, left, right, top, bottom)
np.savetxt("transition.txt",t, fmt="%d")
np.savetxt("ratio.txt",r, fmt="%1.3f")
np.savetxt("normalize.txt",norm ,fmt="%1.3f")
np.savetxt("angles.txt",norm ,fmt="%1.3f")
np.savetxt("centroid.txt",c ,fmt="%s")
img.show()

### The resultant figure will be like as follow: <img src="result.png">

# END